In [2]:
import requests
from dataset import get_recipe
from prompt import example_prompt
import pandas as pd

In [7]:
url = 'http://127.0.0.1:8000/gemini/en/'

In [3]:
recipe = get_recipe(1)

In [4]:
recipe_name = recipe['name']
ingredients = recipe['ingredients']
steps = recipe['steps']
time = recipe['minutes']

In [5]:
data = {
    'recipe_name' : recipe_name,
    'ingredients' : ingredients,
    'steps' : steps,
    'time' : time
}

In [18]:
response = requests.post(url=url, data=data)

In [19]:
response.json()

{'response': '<step 1>\n1. What is the purpose of preheating the oven?\n2. What type of dough is used in this step?\n3. How much dough is required?\n4. What should the thickness of the dough be?\n5. What is the best method for transferring the dough to the baking sheet?\n</step 1>\n<step 2>\n1. What are the ingredients for the parmesan sauce?\n2. How much of each ingredient is required for the parmesan sauce?\n3. What is the best method for preparing the parmesan sauce?\n4. How can you tell when the parmesan sauce is ready?\n5. What is the purpose of adding the parmesan sauce to the dough?\n</step 2>\n<step 3>\n1. What type of cheese is used in this step?\n2. How much cheese is required?\n3. What is the best method for spreading the cheese evenly over the dough?\n4. How long should the dough be baked?\n5. What temperature should the oven be set to?\n</step 3>\n<step 4>\n1. What type of eggs are used in this step?\n2. How many eggs are required?\n3. What is the best method for whisking 